# 준비사항
1. 드라이브 마운트
2. 패키지 설치

In [1]:
#드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 8.0 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1232 sha256=0ce61083be68e4d40a510743d61d7150207505a02b7b6353744ea2f0926af648
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   754k      0  0:00:01  0:00:01 --:--:-- 1151k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example

In [3]:
# 패키지 설치
!pip install kiwipiepy #형태소분석기
!pip install tomotopy #LDA 설계용
import pandas as pd #xlsx파일 불러오기 위한 pandas
import numpy as np
import datetime as dt #날짜 자료형 패키지
import matplotlib.pyplot as plt
import seaborn as sns
import re #정규표현식
import random #랜덤함수
import tomotopy as tp #tomotopy
import collections #counter 호출용
from math import sqrt #제곱근 계산
from tqdm import tqdm
tqdm.pandas() #진행상황 확인
from kiwipiepy import Kiwi
#, Option #키위
from matplotlib import font_manager, rc #시각화 폰트 패키지
from wordcloud import WordCloud,STOPWORDS #워드클라우드 패키지
from PIL import Image #이미지 배경 패키지

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 35.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.16.0-py3-none-any.whl size=30813256 sha256=ed8ee93153a5e52e559d6f6c53d1f94781413ee25ee82228f1a956ba86a8e35c
  Stored in directory: /root/.cache/pip/wheels/6b/03/2b/ac2c97cc65ebd9df3516f4b900adc2f0a744df8d1375b2e2ef
Successfully built kiwipiepy-model
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 11.6 MB/s eta 0:00:00


In [4]:
from konlpy.tag import Mecab
mecab = Mecab()
print(mecab.morphs("사과와 바나나 그리고 포도를 좋아합니다"))

['사과', '와', '바나나', '그리고', '포도', '를', '좋아합니다']


In [5]:
#news_all.to_excel('/content/drive/MyDrive/dscvr_g3_preproc.xlsx',encoding='CP949') #전처리 후 중간 저장
news_all=pd.read_csv('/content/drive/MyDrive/2023 겨울계절 URP/데이터_수집/N번방_네이버뉴스.csv',index_col=0) #저장된 파일 불러오기

In [6]:
#파일 확인
news_all.head()

,link,작성날짜,제목,언론사,댓글수
URL,,,,,
https://n.news.naver.com/mnews/article/016/0001661132?sid=102,https://n.news.naver.com/mnews/article/016/000...,2020-04-13 09:38:05,"검찰, 오늘 조주빈 구속기소… 범죄수익 추징 등 보강수사 계속",헤럴드경제,NaN
https://n.news.naver.com/mnews/article/020/0003278730?sid=100,https://n.news.naver.com/mnews/article/020/000...,2020-04-02 03:04:09,‘박사방’ 영상 재유포자도 수사 착수,동아일보,0
https://n.news.naver.com/mnews/article/011/0003721045?sid=102,https://n.news.naver.com/mnews/article/011/000...,2020-04-07 21:18:03,"경찰, 조주빈 공범 ‘부따’ 구속영장 신청",서울경제,0
https://n.news.naver.com/mnews/article/003/0009861709?sid=102,https://n.news.naver.com/mnews/article/003/000...,2020-05-14 11:07:51,"완장방·주홍글씨 운영자 '미희', 구속심사 16분만 종료(종합)",뉴시스,2
https://n.news.naver.com/mnews/article/020/0003291851?sid=102,https://n.news.naver.com/mnews/article/020/000...,2020-06-14 21:44:02,"해군 대위가 ‘성착취 동영상’ 제작…경찰, 가해자 다수 체포",동아일보,56


In [7]:
news_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5491 entries, https://n.news.naver.com/mnews/article/016/0001661132?sid=102 to https://n.news.naver.com/mnews/article/028/0002493970?sid=102
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    5491 non-null   object
 1   작성날짜    5491 non-null   object
 2   제목      5491 non-null   object
 3   언론사     5491 non-null   object
 4   댓글수     4969 non-null   object
dtypes: object(5)
memory usage: 257.4+ KB


#2-2. 기사 토픽모델링
```
셀 실행 순서
1. 분석 준비
2. 명사 추출 함수 정의
3. 명사 추출
4. 정규화 및 커스텀 딕셔너리 작업
5. 다시 명사 추출
6. 불용어 처리
```


##형태소분석

###분석 준비

In [8]:
#형태소분석기 호출★
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [9]:
#특수기호 제거
pattern1 = '[-=+,#/\?:;^$.A*"~%!\\\n\r\t<>[]{}()▼▽♣♧§☆♡´∀｀ﾉ♥♡▲△�ㅁ▼▽▶▷◀◁★☆♠♤●○◎⊙◆◇■□＊━▣“”]' #특수기호 목록
news_all['제목']=news_all['제목'].progress_map(lambda x:re.sub(pattern1, ' ', x).strip())  # 특수 기호를 " " 공란으로

100%|██████████| 5491/5491 [00:00<00:00, 234863.23it/s]


###명사 추출
kiwi형태소분석은 다음과 같은 형태로 분석이 이루어집니다.

In [10]:
kiwi.analyze('금잔디에 가면 볶음우동을 먹을 수 있어요')

[([Token(form='금잔디', tag='NNP', start=0, len=3),
   Token(form='에', tag='JKB', start=3, len=1),
   Token(form='가', tag='VV', start=5, len=1),
   Token(form='면', tag='EC', start=6, len=1),
   Token(form='볶음', tag='NNG', start=8, len=2),
   Token(form='우동', tag='NNG', start=10, len=2),
   Token(form='을', tag='JKO', start=12, len=1),
   Token(form='먹', tag='VV', start=14, len=1),
   Token(form='을', tag='ETM', start=15, len=1),
   Token(form='수', tag='NNB', start=17, len=1),
   Token(form='있', tag='VA', start=19, len=1),
   Token(form='어요', tag='EF', start=20, len=2)],
  -68.07340240478516)]

kiwi는 명사 추출 기능이 존재하지 않기 때문에 사용자함수를 정의해 주었습니다. 명사와 고유명사를 추출합니다.

In [11]:
 #명사 추출 함수 (kiwi 0.10.0)
 def get_noun(texts):
    res = kiwi.analyze(texts)[0][0]
    return [Token[0]
            for Token in res
            if Token[1]=='NNG' or Token[1]=='NNP']

#함수 동작 체크
get_noun('금잔디에 가면 볶음우동을 먹을 수 있어요')

['금잔디', '볶음', '우동']

**부가설명**

Kiwi는 `List[Tuple[List[Tuple[형태소, 품사태그, 시작위치, 문자열길이]], 분석 점수]]` 형태로 분석 결과를 출력합니다.

맨 바깥 리스트에서 res라는 변수를 생성하고(Line 2), 이를 활용하여 품사 태그가 NNG(명사) 혹은 NNP(고유명사)인 형태소만을
리스트 형태로 반환합니다(Line 3~5).

참고로 Kiwi의 품사 태그는 Sejong의 것과 동일합니다.

In [12]:
# 최초 형태소 분석★
# 전처리 후 다시 형태소분석을 진행할 때도 이 셀을 실행해주세요.
news_all['nouns'] = news_all['제목'].progress_map(lambda x:get_noun(x))

100%|██████████| 5491/5491 [00:13<00:00, 422.15it/s]


In [13]:
#결과 확인
news_all.head()

,link,작성날짜,제목,언론사,댓글수,nouns
URL,,,,,,
https://n.news.naver.com/mnews/article/016/0001661132?sid=102,https://n.news.naver.com/mnews/article/016/000...,2020-04-13 09:38:05,"검찰, 오늘 조주빈 구속기소… 범죄수익 추징 등 보강수사 계속",헤럴드경제,NaN,"[검찰, 오늘, 조, 주빈, 구속, 기소, 범죄, 수익, 추징, 보강, 수사]"
https://n.news.naver.com/mnews/article/020/0003278730?sid=100,https://n.news.naver.com/mnews/article/020/000...,2020-04-02 03:04:09,‘박사방’ 영상 재유포자도 수사 착수,동아일보,0,"[박사, 방, 영상, 재유포자, 수사, 착수]"
https://n.news.naver.com/mnews/article/011/0003721045?sid=102,https://n.news.naver.com/mnews/article/011/000...,2020-04-07 21:18:03,"경찰, 조주빈 공범 ‘부따’ 구속영장 신청",서울경제,0,"[경찰, 조주, 공범, 부따, 구속, 영장, 신청]"
https://n.news.naver.com/mnews/article/003/0009861709?sid=102,https://n.news.naver.com/mnews/article/003/000...,2020-05-14 11:07:51,"완장방·주홍글씨 운영자 '미희', 구속심사 16분만 종료(종합)",뉴시스,2,"[완장, 방, 주홍글씨, 운영자, 미희, 구속, 심사, 종료, 종합]"
https://n.news.naver.com/mnews/article/020/0003291851?sid=102,https://n.news.naver.com/mnews/article/020/000...,2020-06-14 21:44:02,"해군 대위가 ‘성착취 동영상’ 제작…경찰, 가해자 다수 체포",동아일보,56,"[해군, 대위, 성, 착취, 동영상, 제작, 경찰, 가해자, 다수, 체포]"


In [14]:
news_all['nouns']

URL
https://n.news.naver.com/mnews/article/016/0001661132?sid=102          [검찰, 오늘, 조, 주빈, 구속, 기소, 범죄, 수익, 추징, 보강, 수사]
https://n.news.naver.com/mnews/article/020/0003278730?sid=100                            [박사, 방, 영상, 재유포자, 수사, 착수]
https://n.news.naver.com/mnews/article/011/0003721045?sid=102                         [경찰, 조주, 공범, 부따, 구속, 영장, 신청]
https://n.news.naver.com/mnews/article/003/0009861709?sid=102               [완장, 방, 주홍글씨, 운영자, 미희, 구속, 심사, 종료, 종합]
https://n.news.naver.com/mnews/article/020/0003291851?sid=102            [해군, 대위, 성, 착취, 동영상, 제작, 경찰, 가해자, 다수, 체포]
                                                                                       ...                        
https://n.news.naver.com/mnews/article/417/0000535737?sid=102                                 [박사, 방, 조주빈, 공판, 개시]
https://n.news.naver.com/mnews/article/003/0009780580?sid=102                           [방, 박사, 방, 이번, 박멸, 경찰, 명운]
https://n.news.naver.com/mnews/article/056/0010809595?sid=102          [검찰, 

## 텍스트 전처리 반복
다음 과정을 **아주 많이** 반복합니다:

1. 일부 아이템을 랜덤하게 추출하여 확인

2. 정규화

3. 커스텀 사전 추가

4. 불용어 제거

5. 다시 형태소분석

**기사 랜덤추출**

In [15]:
#카운터 리셋
counter=0

In [16]:
i = random.randint(0, 5491)

print("article {}".format(i)) #인덱스 번호
print(news_all.iloc[i][['제목','nouns']].values[0]) #기사 본문
print(news_all.iloc[i][['제목','nouns']].values[1]) #생성된 토큰
counter+=1
print('counter:{}'.format(counter))

article 4086
'텔레그램방 성범죄' 124명 검거·18명 구속…신상공개 사상 최대
['텔레그램', '방', '성범죄', '검거', '구속', '신상', '공개', '사상', '최대']
counter:1


**커스텀 사전 추가**
* 고유명사나 신조어, 한 단어로 묶여야 할 단어를 명사로 추가 지정하여 올바르게 명사로 분류될 수 있도록 해 줘야 함
* 그 외 형태소분석이 이상하게 되는 단어도 수동으로 지정해 주면 좋음

In [17]:
#커스텀 사전★
word_list=['사이버팀','실효성','특별조사팀','AI','아동청소년','영화화','성노예화','아동청소년성보호','정승민','자처','재난지원금','범죄단체조직죄','주민번호'\
    "산·학",'김유빈','창X들','성착취','IP','박사방','능욕방','제2의','대학생','시민방범대','소통령','협박죄','예비후보','엄중처벌','교사단체','공인요원','신상공개','주거침입강제추행죄','여성의당','통합당',
           '불법촬영물','박사방','신상공개','그루밍','성범죄물','물차별','미성년자','착취범','처벌강화법','무법천지','종족주의','2차','양성희','위장수사','제2','재발','비윤리성','문형욱','무기징역','내주',\
           'n번방','N번방','국민여론','무기징역형','유료회원','아·청','그알','최진성','무죄','국민참여재판','재판','범죄추적극','엘','N번방','국민청원','오덕식',\
           '성착취물','단순소지','디지털성범죄물','방지방법원','국무회의','촬영물','SNS','성평등선대본','임시국회','박사방','조직죄','범죄단체','조주빈','개설자','경찰조사','갓갓',\
           '심바','강력처벌','JMS','처벌촉구','신상공개','디지털성범죄','법정최고형','반일','이우연','양형기준','솜방망이처벌','나체사진','불법촬영물','기자회견','여성후보','상무위원회','동물판']

In [18]:
#커스텀 사전 리스트를 dataframe형태로 변환
custom_dict = pd.DataFrame({"word":word_list})

#품사태그는 고유명사로 지정
custom_dict['morp']="NNP"

#Kiwi는 우선순위를 부여하기 위해 score개념이 존재합니다. 커스텀 단어를 우선 활용하기 위해 score값에 1을 부여.
custom_dict['score']=1

#데이터 확인
custom_dict.head()

,word,morp,score
0,사이버팀,NNP,1
1,실효성,NNP,1
2,특별조사팀,NNP,1
3,AI,NNP,1
4,아동청소년,NNP,1


In [19]:
#커스텀 사전을 csv로 저장하고 불러옴★
custom_dict.to_csv("/content/drive/MyDrive/2023 겨울계절 URP/데이터_수집/custom_dict.txt", index=False, header=False, sep="\t",encoding='UTF-8')

In [20]:
#kiwi의 load_user_dictionary는 커스텀 사전을 변경할 경우 업데이트된 단어의 수를 반환해 줍니다.
kiwi.load_user_dictionary('/content/drive/MyDrive/2023 겨울계절 URP/데이터_수집/custom_dict.txt')

91

**단어 정규화**
* 정규화: 동일한 표현을 다르게 쓴 것을 모두 통일해주는 것

영어 약자, 유행병 이름, 사람 이름, 한자 및 영어 표현 등 수작업으로 교체하여 분류 정확도 높이기

In [21]:
#정규화 딕셔너리 ★
dic_standardization={'法':'법원','검·경':' 檢·警','디지털성범죄':'디지털성범죄물','성착취':'성착취물','中':'중국','靑':'청와대','文':'문','★':'연예인','무기징역형':'무기징역','女':'여자',\
                     '그알':'그것이알고싶다','n번방':'N번방','군':'軍','검사':'檢',\

                     '감사위':'감사위원회','고리':'고리원전','고검':'고등검찰','고법':'고등법원','공수처':'고위공직자범죄수사처','과기정통부':'과학기술정보통신부',\
                     '과방위':'과학기술정보방송통신위원회','국토부':'국토교통부','국편':'국사편찬위원회','국힘':'국민의힘','개인정보범죄 정부합동수사단': '개인정보범죄정부합동수사단',\
                     '기재부':'기획재정부','대검':'대검찰청','대법':'대법원','대사':'외교관','도카이제2원전':'도카이원전','리스크':'위험 요인','모럴 해저드':'도덕적 해이','방폐물':'방사성폐기물',\
                     '방사성 폐기물':'방사성폐기물', '박 대통령':'박근혜 대통령','산업부':'산업통상자원부','시민당':'더불어시민당','시 주석':'시진핑 국가주석', '사용후 핵연료':'사용후핵연료',\
                     '산업자원통상부':'산업통상자원부','식약처':'식품의약품안전처','신고리':'신고리원전', '신한울':'신한울원전','여수산단':'여수국가산업단지','영구저장시설': '영구처분시설',\
                     '산업부':'산업통상자원부','산자위':'산업통상자원중소벤처기업위원회','안전처': '국민안전처','영덕 원전':'천지원전', '영덕 원전':'영덕 천지원전','원안위':'원자력안전위원회',\
                     '아베 신조': '아베', '오이': '오이원전','월성본부':'월성원자력본부','원전':'원자력발전소', '원해연':'원자력해체기술종합연구센터',\
                     '원자력시설 해체기술 종합연구센터':'원자력해체기술종합연구센터','인공 태양':'인공태양','즉시 해체':'즉시해체','지법':'지방법원', '지검':'지방검찰청','지연 해체':'지연해체',\
                     '체르노빌 발전소': '체르노빌 원전','트리튬':'삼중수소','특위':'특별위원회','핵발전소':'원자력발전소','한수원':'한국수력원자력',  '한빛':'한빛원전','한울':'한울원전',\
                     '핵폐기물':'방사성폐기물','IAEA':'국제원자력기구','NRC':'원자력규제위원회','UAE':'아랍에미리트','韓':'한국','與':'여당','野':'야당','北':'북한','(脫)':'','(非)':'','(正常)':''}

In [22]:
#정규화 딕셔너리를 활용하여 단어 교체★
for old, new in dic_standardization.items():
  print("convert {} into {}.".format (old, new))
  news_all['제목'] = news_all['제목'].progress_map(lambda x:x.replace(old, new))

convert 法 into 법원.


100%|██████████| 5491/5491 [00:00<00:00, 496227.77it/s]


convert 검·경 into  檢·警.


100%|██████████| 5491/5491 [00:00<00:00, 497761.42it/s]


convert 디지털성범죄 into 디지털성범죄물.


...

100%|██████████| 5491/5491 [00:00<00:00, 537402.54it/s]


convert (非) into .


100%|██████████| 5491/5491 [00:00<00:00, 503826.64it/s]


convert (正常) into .


100%|██████████| 5491/5491 [00:00<00:00, 523453.87it/s]


**불용어 제거**

불용어는 분석에 사용되지 않을 단어를 의미합니다. 다음 유형의 단어를 제거하였습니다:

1. 자연어처리 시 통용되는 불용어 리스트
2. 너무 많이 등장하는 단어들 - 모든 문서에서 공통적으로 등장하여 분류 성능을 낮출 우려가 있음.
3. 한 단어로 된 단어
4. 토픽 특성상 핵심 주제어인 '원전' '원자력발전'도 제거하는 게 좋음


In [23]:
#1차 필터링: 한국어 불용어 사전 활용★
stop_list1 = open('/content/drive/MyDrive/2023 겨울계절 URP/코드/재희/토픽모델링/stopword.txt', encoding = 'utf-8')


lines1 = stop_list1.readlines() #한 줄마다 읽어옴


stopwords = [] #불용어 리스트
for line in lines1:
    a= line.split() #줄 단위 분할
    stopwords.append(a) #한 줄마다 불용어 리스트에 추가


In [24]:
i = random.randint(0, 219) #랜덤 아티클 인덱스_진주
#i = random.randint(13484, 26968)) #랜덤 아티클 인덱스_이훈
#i = random.randint(26968, len(news_all)) #랜덤 아티클 인덱스_석호
print("article {}".format(i)) #인덱스 번호
print(news_all.iloc[i][['제목','nouns']].values[0]) #기사 본문
print(news_all.iloc[i][['제목','nouns']].values[1]) #생성된 토큰
counter+=1
print('counter:{}'.format(counter))

article 89
‘박사방’ 조주빈, 항소심 징역 42년 선고
['박사', '방', '조', '주빈', '항소심', '징역', '선고']
counter:2


In [25]:
#2차 필터링: 커스텀 불용어 리스트★
custom_stopwords = ['뉴스하이킥','뉴스큐','이슈와치','단독','초점','이슈','관련','한국','이날','발생','이후','이번','오후','오전','가운데','기자','사진','최근','정부','나이트포커스','정승민']
stopwords+=custom_stopwords #커스텀 불용어를 리스트에 추가

In [26]:
#'불용어 리스트에 없는'형태소만 남기는 방식으로 불용어 제거★
news_all['nouns'] = news_all['nouns'].progress_map(lambda x:[w for w in x if w not in stopwords])

100%|██████████| 5491/5491 [00:00<00:00, 10979.09it/s]


In [27]:
#3차 필터링: 한 글자 제거★
news_all['nouns'] = news_all['nouns'].progress_map(lambda x:[w for w in x if len(w) > 1])

100%|██████████| 5491/5491 [00:00<00:00, 349833.27it/s]


In [28]:
news_all.head()

,link,작성날짜,제목,언론사,댓글수,nouns
URL,,,,,,
https://n.news.naver.com/mnews/article/016/0001661132?sid=102,https://n.news.naver.com/mnews/article/016/000...,2020-04-13 09:38:05,"검찰, 오늘 조주빈 구속기소… 범죄수익 추징 등 보강수사 계속",헤럴드경제,NaN,"[검찰, 오늘, 주빈, 구속, 기소, 범죄, 수익, 추징, 보강, 수사]"
https://n.news.naver.com/mnews/article/020/0003278730?sid=100,https://n.news.naver.com/mnews/article/020/000...,2020-04-02 03:04:09,‘박사방’ 영상 재유포자도 수사 착수,동아일보,0,"[박사, 영상, 재유포자, 수사, 착수]"
https://n.news.naver.com/mnews/article/011/0003721045?sid=102,https://n.news.naver.com/mnews/article/011/000...,2020-04-07 21:18:03,"경찰, 조주빈 공범 ‘부따’ 구속영장 신청",서울경제,0,"[경찰, 조주, 공범, 부따, 구속, 영장, 신청]"
https://n.news.naver.com/mnews/article/003/0009861709?sid=102,https://n.news.naver.com/mnews/article/003/000...,2020-05-14 11:07:51,"완장방·주홍글씨 운영자 '미희', 구속심사 16분만 종료(종합)",뉴시스,2,"[완장, 주홍글씨, 운영자, 미희, 구속, 심사, 종료, 종합]"
https://n.news.naver.com/mnews/article/020/0003291851?sid=102,https://n.news.naver.com/mnews/article/020/000...,2020-06-14 21:44:02,"해軍 대위가 ‘성착취물 동영상’ 제작…경찰, 가해자 다수 체포",동아일보,56,"[해군, 대위, 착취, 동영상, 제작, 경찰, 가해자, 다수, 체포]"


불용어 처리를 마쳤으면, 빈도 수 파악 후 2차 필터링할 단어 리스트를 수정하기 위해, 단어 개수를 카운트합니다.

In [29]:
#단어 빈도 파악
nouncounter=news_all['nouns'].sum() #형태소 목록 병합
counts = collections.Counter(nouncounter) #병합된 목록 카운트
print(counts) #결과

Counter({'사건': 695, '처벌': 525, '텔레그램': 510, '착취': 497, '공개': 483, '박사': 470, '신상': 361, '방지법': 323, '성범죄': 315, '경찰': 300, '징역': 297, '수사': 287, '주빈': 282, '디지털': 267, '공범': 257, '청원': 232, '국회': 230, '피해자': 229, '갓갓': 221, '범죄': 217, '종합': 211, '구속': 198, '검거': 187, '문형': 183, '국민': 182, '가해자': 181, '여성': 175, '운영자': 174, '방지': 159, '회원': 157, '논란': 152, '추적': 148, '검찰': 140, '촉구': 140, '영상': 130, '개설': 126, '전원': 122, '분노': 120, '조사': 108, '통과': 106, '불법': 101, '유포': 99, '뉴스': 99, '얼굴': 97, '처리': 96, '발언': 93, '황교안': 91, '검열': 90, '대통령': 88, '아동': 87, '가능': 86, '촬영': 83, '혐의': 83, '영장': 78, '정치': 74, '주범': 73, '단체': 73, '피해': 72, '통합': 72, '강화': 72, '승진': 71, '윤석열': 71, '조주': 70, '구형': 70, '강력': 69, '음란물': 68, '필요': 67, '운영': 66, '대책': 65, '켈리': 65, '호기심': 63, '사람': 62, '확정': 62, '피의자': 61, '제작': 60, '결정': 60, '속보': 60, '재판': 60, '추가': 60, '본회의': 60, '오늘': 59, '포토': 59, '여야': 59, '이준석': 59, '선고': 59, '청소년': 59, '남성': 58, '최초': 57, '이유': 57, '입법': 56, '법안': 56, '정의당': 55, '협박': 55, '성

카운트 결과는 여기서 보는 것보다 복사해서 메모장이나 워드로 보는게 편해요!

## 토픽모델링

먼저 LDA 모델을 생성합니다.

In [30]:
# LDA Model 정의★
def lda(k_model, iteration, text, word_remove=0):

    # 모델 설정
    model = tp.LDAModel(k=k_model, rm_top=word_remove, seed=29672)

    # 문서 추가
    for n, line in enumerate(text):
        line = ','.join(line)
        model.add_doc(line.strip().split(','))

    model.burn_in = 100
    model.train(0)

    print('문서 개수:', len(model.docs), ', 단어 개수:', len(model.used_vocabs), ', 단어의 총수:', model.num_words, '토픽 수:',k_model)
    #print('제거된 단어들:', model.removed_top_words)

    print('훈련 중',flush=True)
    for i in range(0, iteration, 50):
        model.train(10)

    model.summary()
    return(model)

여러 번 반복을 통해 최적의 K값을 찾습니다. 먼저 K값 후보 리스트를 생성합니다.

In [31]:
'''#Perplexity 값 시각화
fig, ((ax1))= plt.subplots(nrows=1, ncols=1)   #(1,1)짜리 subplot
fig.set_size_inches(8,8)  #사이즈 지정
sns.lineplot(x="k", y="model_perplexity", data=perplexity_df, ax=ax1) #k값과 perplexity를 x, y로 하는 선 그래프

SyntaxError: incomplete input (<ipython-input-31-1842e80077db>, line 1)

K값 정한 후 최적의 모델을 출력합니다.
Perplexity값이 충분히 낮아야 분류 결과의 신뢰가 있지만, K값이 너무 커지면 지나치게 상세하게 분류가 이루어지기 때문에, 적당한 수준에서 K값을 선정하였습니다.



# k값 찾기 (Perplexity)



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_perplexity(cv, start=10, end=30, max_iter=5, topic_word_prior= 0.1,
                    doc_topic_prior=1.0):
    iter_num = []
    per_value = []

    for i in range(start, end + 1):
        lda = LatentDirichletAllocation(n_components = i, max_iter=max_iter,
                                        topic_word_prior= topic_word_prior,
                                        doc_topic_prior=doc_topic_prior,
                                        learning_method='batch', n_jobs= -1,
                                        random_state=7)
        lda.fit(cv)
        iter_num.append(i)
        pv = lda.perplexity(cv)
        per_value.append(pv)
        print(f'n_components: {i}, perplexity: {pv:0.3f}')

    plt.plot(iter_num, per_value, 'g-')
    plt.show()
    return start + per_value.index(min(per_value))

print("n_components with minimum perplexity:",
      show_perplexity(review_cv, start=6, end=15))

In [ ]:
  #최적화된 모델 출력
  k = 30
  mdl_k=lda(k, 1000, text=news_all['nouns']) #모델링

# 문서분류

가장 높은 확률에 해당하는 토픽으로 각 문서를 분류합니다.

In [ ]:
#개별 주제에 속할 확률을 제시하는 데이터프레임
new_df = pd.DataFrame()
for i, line in enumerate(mdl_k.docs):
    temp=pd.DataFrame(line.get_topic_dist()).T
    new_df=new_df.append(temp)
new_df = new_df.reset_index()
new_df = new_df.drop(['index'], axis=1)
new_df.columns = ['#'+ str(x) for x in range(1,k+1)]

In [ ]:
new_df.head()

In [ ]:
#idxmax 활용하여 가장 가능성 높은 확률의 토픽을 기존 데이터 끝에 붙임
theta_df = news_all.merge(new_df, left_index=True, right_index=True)
theta_df['topic']=theta_df[['#'+ str(s) for s in range(1, k+1)]].idxmax(axis=1)
theta_df.head()

In [ ]:
#토픽별로 최빈 단어 파악하기 위한 데이터프레임
topic_df = pd.DataFrame()
for i in range(0, k):
  temp = pd.DataFrame()
  temp = pd.DataFrame(mdl_k.get_topic_words(i, top_n=10))
  temp.columns = ["#"+str(i+1),"probs"+str(i+1)]
  temp = temp.reset_index()
  if (i==0):
      topic_df = topic_df.append(temp, ignore_index=True)
  else:
      topic_df = topic_df.merge(temp, left_on="index", right_on="index")

최종 토픽 확인

In [ ]:
#토픽 확인하기
for i in range(1,31):
  cnt = len(theta_df[theta_df['topic']=='#'+str(i)])
  print("토픽 {:8}: 토픽 개수 {:8}, 전체 퍼센트 {:8.3f}%".format(i, cnt, cnt/len(news_all)*100)) # 해당 토픽의 정보
  print(topic_df['#'+str(i)]) # 해당 토픽에서 가장 많이 나오는 단어



  print(theta_df[theta_df['topic']=='#'+str(i)]['제목'].sample(n=20).values) # 임의로 선택 20(n)개의 문서
  print("\n\n==================================================================================\n\n")

In [ ]:
theta_df.head()

## 분석을 위한 데이터프레임 가공

In [ ]:
#데이터프레임 피봇
groupby_topic=theta_df[['id','press','topic']].groupby(['press','topic']).agg('count')
group_pivot=groupby_topic.pivot_table(index='press',columns='topic')
group_pivot=group_pivot['id']

In [ ]:
#토픽 번호 데이터프레임 생성
df=[]
for i in range(1,31):
  df.append('#'+str(i))
group_pivot=group_pivot[df]

In [ ]:
#토픽별 집계
groupby_topic=theta_df[['id','press','topic']].groupby(['press','topic']).agg('count')

In [ ]:
groupby_topic

In [ ]:
group_pivot

In [ ]:
df=[]
for i in range(1,31):
  df.append('#'+str(i))
group_pivot=group_pivot[df]

In [ ]:
group_pivot.to_csv('/content/drive/MyDrive/dscvr_g3/dscvr_g3_pivot_df.xlsx',encoding='CP949') #내보내기